In [16]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.applications import VGG16
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from keras_tuner import HyperModel, RandomSearch
from tensorflow.keras.metrics import AUC
from sklearn.model_selection import train_test_split
from skimage.io import imread
from skimage.transform import resize
import numpy as np
import os
import matplotlib.pyplot as plt

In [17]:
def load_images(Categories, datadir):
    flat_data_arr=[]
    target_arr=[]
    for i in Categories:
        print(f'loading... category : {i}')
        path=os.path.join(datadir,i)
        for img in os.listdir(path):
            if img.endswith(".jpg") or img.endswith(".png"):  
                img_array=imread(os.path.join(path,img))
                img_resized=resize(img_array,(150,150,3))
                flat_data_arr.append(img_resized.flatten())
                target_arr.append(Categories.index(i))
        print(f'loaded category:{i} successfully')
    flat_data=np.array(flat_data_arr)
    target=np.array(target_arr)
    return flat_data, target

In [18]:
class CNNHyperModel(HyperModel):
    def __init__(self, input_shape, num_classes):
        self.input_shape = input_shape
        self.num_classes = num_classes

    def build(self, hp):
        base_model = VGG16(include_top=False, weights='imagenet', input_shape=self.input_shape)
        model = Sequential()
        model.add(base_model)
        model.add(Flatten())
        model.add(Dense(units=hp.Int('units',
                                     min_value=32,
                                     max_value=512,
                                     step=32,
                                     default=128),
                   activation=hp.Choice(
                       'dense_activation',
                       values=['relu', 'tanh', 'sigmoid'],
                       default='relu')))
        model.add(Dense(self.num_classes, activation='softmax'))

        model.compile(
            optimizer=keras.optimizers.SGD(
                hp.Choice('learning_rate',
                          values=[1e-2, 1e-3, 1e-4])),
            loss='categorical_crossentropy',
            metrics=['accuracy', AUC(name='auc')])
        return model

In [19]:
# Data Augmentation
def create_datagen():
    datagen = ImageDataGenerator(
        featurewise_center=True,
        featurewise_std_normalization=True,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True)
    return datagen

In [20]:
def train_model(tuner, X_train, y_train, X_val, y_val, datagen, epochs):
    stop_early = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    tuner.search(datagen.flow(X_train, y_train, batch_size=32), 
                 validation_data=(X_val, y_val), 
                 epochs=epochs, callbacks=[stop_early])


def get_best_model(tuner):
    return tuner.get_best_models(num_models=1)[0]


def evaluate_model(model, X_test, y_test):
    loss, accuracy = model.evaluate(X_test, y_test)
    print(f"Test accuracy: {accuracy}")
    print(f"Test loss: {loss}")


def model_pipeline(X, y, input_shape, num_classes, epochs=20):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=77, stratify=y)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=77, stratify=y_train)
    X_train = X_train.reshape(-1, 150, 150, 3)
    X_val = X_val.reshape(-1, 150, 150, 3)
    X_test = X_test.reshape(-1, 150, 150, 3)
    X_train = X_train / 255.0
    X_val = X_val / 255.0
    X_test = X_test / 255.0
    datagen = create_datagen()
    datagen.fit(X_train)
    hypermodel = CNNHyperModel(input_shape, num_classes)
    tuner = RandomSearch(hypermodel, objective='val_accuracy', max_trials=10, seed=42, directory='random_search')
    tuner.search_space_summary()
    train_model(tuner, X_train, y_train, X_val, y_val, datagen, epochs)
    model = get_best_model(tuner)
    evaluate_model(model, X_test, y_test)

In [21]:
def plot_history(history):
    plt.figure(figsize=(10,4))
    plt.subplot(1,2,1)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epochs')
    plt.legend(['Train', 'Validation'])

    plt.subplot(1,2,2)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epochs')
    plt.legend(['Train', 'Validation'])
    plt.tight_layout()
    plt.show()


In [22]:
# Load the data
Categories=['covid','normal']
datadir='dataset_18/'
X, y = load_images(Categories, datadir)
y = to_categorical(y)  # one-hot encode the labels

input_shape = (150, 150, 3)
num_classes = 2

# Run the pipeline
model_pipeline(X, y, input_shape, num_classes)

# Later, after running the search
tuner.results_summary()

# For the best model's summary:
best_model = tuner.get_best_models(1)[0]
best_model.summary()

Trial 9 Complete [00h 12m 24s]
val_accuracy: 0.5882353186607361

Best val_accuracy So Far: 0.5882353186607361
Total elapsed time: 00h 48m 54s

Search: Running Trial #10

Value             |Best Value So Far |Hyperparameter
352               |352               |units
sigmoid           |relu              |dense_activation
0.01              |0.001             |learning_rate



Epoch 1/20
9/9 [==============================] - 45s 5s/step - loss: 0.7014 - accuracy: 0.5830 - auc: 0.6099 - val_loss: 0.7146 - val_accuracy: 0.5882 - val_auc: 0.5882
Epoch 2/20
9/9 [==============================] - 44s 5s/step - loss: 0.6500 - accuracy: 0.6642 - auc: 0.7038 - val_loss: 0.7037 - val_accuracy: 0.4118 - val_auc: 0.4118
Epoch 3/20
9/9 [==============================] - 48s 5s/step - loss: 0.6169 - accuracy: 0.6900 - auc: 0.7472 - val_loss: 0.8947 - val_accuracy: 0.4118 - val_auc: 0.4118
Epoch 4/20
9/9 [==============================] - 43s 5s/step - loss: 0.5510 - accuracy: 0.7232 - auc: 0.8067 - val_loss: 1.1383 - val_accuracy: 0.4118 - val_auc: 0.4118
Epoch 5/20
9/9 [==============================] - 43s 5s/step - loss: 0.5594 - accuracy: 0.7565 - auc: 0.7889 - val_loss: 0.8924 - val_accuracy: 0.4118 - val_auc: 0.4118
Epoch 6/20
9/9 [==============================] - 43s 5s/step - loss: 0.6368 - accuracy: 0.7380 - auc: 0.7601 - val_loss: 0.9611 - val_accuracy: 0.411